## 0. Load raw Airbnb listings and inspect structure

We start by loading the raw `listings.csv` file and quickly checking:
- number of rows and columns
- datatypes
- which columns are mostly missing

This gives us a first idea of the cleaning work needed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Loading the raw file
df = pd.read_csv('/content/drive/MyDrive/Projet Machine Learning/Final version/1) Scraped from Airbnb listings.csv')

# Overview of the structure of the raw dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96871 entries, 0 to 96870
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            96871 non-null  int64  
 1   listing_url                                   96871 non-null  object 
 2   scrape_id                                     96871 non-null  int64  
 3   last_scraped                                  96871 non-null  object 
 4   source                                        96871 non-null  object 
 5   name                                          96871 non-null  object 
 6   description                                   94421 non-null  object 
 7   neighborhood_overview                         41208 non-null  object 
 8   picture_url                                   96865 non-null  object 
 9   host_id                                       96871 non-null 

## 1. Keep only listings with a price and drop fully empty columns

We keep only rows where the `price` is present (rows without price are useless
for a pricing model), and we remove columns that are completely empty.

This gives us a first “cleaned” version called `df_cleaned`.


In [ ]:
# Keep only lines with a price entered
df_price_clean = df.dropna(subset=['price'])

# Remove completely empty columns
df_cleaned = df_price_clean.dropna(axis=1, how='all')


## 2. Clean `price` and `host_acceptance_rate` formats

The `price` column is stored as text (e.g. `"$120"`).  
We:
- remove currency symbols and thousands separators,
- convert it to numeric.

We also convert `host_acceptance_rate` (e.g. `"96%"`) to a numeric ratio between 0 and 1.


In [ ]:
# Cleaning the price column: removing symbols and converting to float
df_cleaned['price'] = (
    df_cleaned['price']
        .str.replace(r'[$,]', '', regex=True)
        .astype(float)
)

# Initial conversion of host_acceptance_rate to a ratio between 0 and 1
df_cleaned['host_acceptance_rate'] = (
    df_cleaned['host_acceptance_rate']
        .str.replace('%', '')
        .astype(float) / 100
)


/tmp/ipython-input-696108097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['price'] = (
/tmp/ipython-input-696108097.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['host_acceptance_rate'] = (


## 3. Select useful columns and handle missing values on key variables

From `df_cleaned`, we drop identifiers, URLs, textual description fields and
other columns not useful for modelling.

Then, on the remaining dataset `df_clean`:
- we **drop rows** where core structural fields (`bedrooms`, `beds`, `bathrooms`)
  are missing,
- we **impute missing review score fields with 0**, meaning “no score / no reviews yet”.


In [ ]:
# Initial selection of columns relevant for analysis
cols_to_drop = [
    'id', 'scrape_id', 'listing_url', 'host_url', 'picture_url',
    'host_thumbnail_url', 'host_picture_url', 'last_scraped',
    'calendar_last_scraped', 'neighbourhood', 'host_neighbourhood',
    'bathrooms_text', 'name', 'source', 'description',
    'neighborhood_overview','host_id', 'host_name', 'host_location',
    'host_about', 'latitude', 'longitude',
    'host_response_rate', 'availability_60', 'availability_90'
]
df_clean = df_cleaned.drop(columns=cols_to_drop)

# Just to visualize the cleaned neighborhood column
df_cleaned['neighbourhood_cleansed']

# Removal of lines without major structural information
df_clean.dropna(subset=['bedrooms', 'beds', 'bathrooms'], inplace=True)

# Review score columns to be set to 0
score_cols = [
    'reviews_per_month',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

# Imputation of missing values to 0 for scores
df_clean[score_cols] = df_clean[score_cols].fillna(0)

# Overview of the new DataFrame
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 61649 entries, 0 to 96870
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_since                                    61624 non-null  object 
 1   host_response_time                            55414 non-null  object 
 2   host_acceptance_rate                          56778 non-null  float64
 3   host_is_superhost                             60297 non-null  object 
 4   host_listings_count                           61624 non-null  float64
 5   host_total_listings_count                     61624 non-null  float64
 6   host_verifications                            61624 non-null  object 
 7   host_has_profile_pic                          61624 non-null  object 
 8   host_identity_verified                        61624 non-null  object 
 9   neighbourhood_cleansed                        61649 non-null  obje

## 4. Drop additional low-value columns and remove remaining missing values

We remove additional columns that are either:
- derived KPIs we don't want to predict directly,
- or columns that are too detailed / redundant.

Then we drop the remaining rows with missing values to obtain a fully
filled dataset (`df_clean`) for the next feature engineering steps.


In [ ]:
list_to_drop = [
    'estimated_revenue_l365d',
    'estimated_occupancy_l365d',
    'availability_eoy',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'source',
    'name',
    'description',
    'neighborhood_overview',
    'host_about',
    'host_id',
    'host_name',
    'host_location',
    'host_verifications',
    'host_listings_count',
    'host_total_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'first_review',
    'last_review',
    'has_availability',
    'beds',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'number_of_reviews_ly',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]
df_clean = df_clean.drop(columns=list_to_drop, errors='ignore')

# Remove rows that still have NaNs
df_clean.dropna(inplace=True)

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53086 entries, 0 to 96870
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_since                      53086 non-null  object 
 1   host_response_time              53086 non-null  object 
 2   host_acceptance_rate            53086 non-null  float64
 3   host_is_superhost               53086 non-null  object 
 4   host_has_profile_pic            53086 non-null  object 
 5   host_identity_verified          53086 non-null  object 
 6   neighbourhood_cleansed          53086 non-null  object 
 7   property_type                   53086 non-null  object 
 8   room_type                       53086 non-null  object 
 9   accommodates                    53086 non-null  int64  
 10  bathrooms                       53086 non-null  float64
 11  bedrooms                        53086 non-null  float64
 12  amenities                       53086

## 5. Convert textual booleans (t/f) to numeric 0–1

Several host-related columns are stored as `'t'` / `'f'`.  
We convert them to integers (1 = True, 0 = False) so they can be used by ML models.


In [ ]:
binary_cols = [
    'host_is_superhost',
    'host_has_profile_pic',
    'host_identity_verified',
    'instant_bookable'
]

for col in binary_cols:
    df_clean[col] = df_clean[col].map({'t': 1, 'f': 0}).astype(int)

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53086 entries, 0 to 96870
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_since                      53086 non-null  object 
 1   host_response_time              53086 non-null  object 
 2   host_acceptance_rate            53086 non-null  float64
 3   host_is_superhost               53086 non-null  int64  
 4   host_has_profile_pic            53086 non-null  int64  
 5   host_identity_verified          53086 non-null  int64  
 6   neighbourhood_cleansed          53086 non-null  object 
 7   property_type                   53086 non-null  object 
 8   room_type                       53086 non-null  object 
 9   accommodates                    53086 non-null  int64  
 10  bathrooms                       53086 non-null  float64
 11  bedrooms                        53086 non-null  float64
 12  amenities                       53086

## 6. Engineer amenities features (`num_amenities` & key amenity flags)

The `amenities` column is a long text string describing equipment.
We:

1. Convert each string into a **Python list** of amenities.
2. Create `num_amenities` = number of amenities per listing.
3. Create binary flags for **critical amenities** (Wifi, Kitchen, Washer, etc.).
4. Drop the original `amenities` text columns, keeping only numeric/binary features.


In [ ]:
# Function to convert the string into a list of equipment
def clean_amenities(amenities_str):
    if isinstance(amenities_str, str):
        cleaned = amenities_str.strip('[]').replace('"', '')
        return [item.strip() for item in cleaned.split(',') if item.strip()]
    return []

# Apply conversion to list
df_clean['amenities_list'] = df_clean['amenities'].apply(clean_amenities)

# Total number of pieces of equipment
df_clean['num_amenities'] = df_clean['amenities_list'].apply(len)

# Key equipment often linked to price
critical_amenities = [
    'Wifi',
    'Kitchen',
    'Washer',
    'Dryer',
    'Self check-in',
    'Air conditioning',
    'Heating',
    'Private entrance',
    'TV'
]

for amenity in critical_amenities:
    col_name = f'has_{amenity.lower().replace(" ", "_")}'
    df_clean[col_name] = df_clean['amenities_list'].apply(
        lambda x: 1 if amenity in x else 0
    )

# Removal of intermediate columns
df_clean.drop(columns=['amenities', 'amenities_list'], inplace=True)

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53086 entries, 0 to 96870
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_since                      53086 non-null  object 
 1   host_response_time              53086 non-null  object 
 2   host_acceptance_rate            53086 non-null  float64
 3   host_is_superhost               53086 non-null  int64  
 4   host_has_profile_pic            53086 non-null  int64  
 5   host_identity_verified          53086 non-null  int64  
 6   neighbourhood_cleansed          53086 non-null  object 
 7   property_type                   53086 non-null  object 
 8   room_type                       53086 non-null  object 
 9   accommodates                    53086 non-null  int64  
 10  bathrooms                       53086 non-null  float64
 11  bedrooms                        53086 non-null  float64
 12  price                           53086

## 7. Encode `host_response_time` as an ordered numeric variable

We transform the textual response time (`'within an hour'`, `'within a few hours'`, etc.)
into an ordinal score (4 = best, 1 = slowest).

Then we drop the original text column.


In [ ]:
df_clean['host_response_time']

# Ordinal mapping for host_response_time
response_time_mapping = {
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}

df_clean['host_response_time_encoded'] = df_clean['host_response_time'].map(response_time_mapping)
df_clean.drop(columns=['host_response_time'], inplace=True)


### 7bis. Quick sanity-check of unique values per column

We print unique values and examples for each column to validate the
transformation choices and detect any weird values.


In [ ]:
for column in df_clean.columns:
    unique_values = df_clean[column].unique()
    print(f"Column : {column}")
    print(f"Number of unique values : {len(unique_values)}")
    print(f"Examples of values : {unique_values[:10]}")
    print("-" * 50)


Column : host_since
Number of unique values : 5189
Examples of values : ['2009-11-16' '2010-01-04' '2010-05-27' '2010-06-30' '2010-07-04'
 '2010-07-13' '2010-07-15' '2010-07-27' '2010-08-01' '2010-08-29']
--------------------------------------------------
Column : host_acceptance_rate
Number of unique values : 101
Examples of values : [0.96 0.88 0.98 0.91 1.   0.92 0.57 0.9  0.97 0.94]
--------------------------------------------------
Column : host_is_superhost
Number of unique values : 2
Examples of values : [1 0]
--------------------------------------------------
Column : host_has_profile_pic
Number of unique values : 2
Examples of values : [1 0]
--------------------------------------------------
Column : host_identity_verified
Number of unique values : 2
Examples of values : [1 0]
--------------------------------------------------
Column : neighbourhood_cleansed
Number of unique values : 33
Examples of values : ['Islington' 'Westminster' 'Tower Hamlets' 'Richmond upon Thames'
 'Har

## 8. Robust cleaning of `host_acceptance_rate` (safety check)

We apply a defensive cleaning step on `host_acceptance_rate`:
- if it is still of type `object`, we strip `%` and convert to float,
- if the scale is 0–100 we rescale it to 0–1.

In practice it is already clean from Phase 2, but this ensures robustness.


In [ ]:
col_name = 'host_acceptance_rate'

# Step 1: robust conversion if still in text format
if df_clean[col_name].dtype == 'object':
    df_clean[col_name] = (
        df_clean[col_name]
        .astype(str)
        .str.replace('%', '', regex=False)
        .astype(float)
    )

# Step 2: Possible rescaling (0–1)
if df_clean[col_name].max() > 1.0:
    df_clean[col_name] = df_clean[col_name] / 100.0

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53086 entries, 0 to 96870
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_since                      53086 non-null  object 
 1   host_acceptance_rate            53086 non-null  float64
 2   host_is_superhost               53086 non-null  int64  
 3   host_has_profile_pic            53086 non-null  int64  
 4   host_identity_verified          53086 non-null  int64  
 5   neighbourhood_cleansed          53086 non-null  object 
 6   property_type                   53086 non-null  object 
 7   room_type                       53086 non-null  object 
 8   accommodates                    53086 non-null  int64  
 9   bathrooms                       53086 non-null  float64
 10  bedrooms                        53086 non-null  float64
 11  price                           53086 non-null  float64
 12  minimum_nights                  53086

## 9. Simplify property types and one-hot encode room/property categories

The original `property_type` has many values.  
We:

1. Simplify it into 5 broader categories: `Apartment`, `House`, `Hospitality`,
   `Specialty`, `Other`.
2. One-hot encode `room_type`.
3. One-hot encode the simplified property type.

This keeps important information while limiting the number of dummy variables.


In [ ]:
df_clean['property_type'].unique()

# Property type simplification function
def simplify_property_type(type_str):
    if 'room' in type_str:
        type_str = type_str.split(' in ')[-1].strip()

    if any(keyword in type_str for keyword in ['apartment', 'unit', 'condo', 'serviced apartment', 'aparthotel']):
        return 'Apartment'
    elif any(keyword in type_str for keyword in ['house', 'home', 'townhouse', 'villa', 'cottage', 'bungalow']):
        return 'House'
    elif any(keyword in type_str for keyword in ['hostel', 'hotel', 'guesthouse', 'bed and breakfast']):
        return 'Hospitality'
    elif any(keyword in type_str for keyword in ['loft', 'guest suite', 'cabin', 'tiny home', 'boat', 'tent', 'sheph']):
        return 'Specialty'
    else:
        return 'Other'

# Apply simplification
df_clean['property_type_simplified'] = df_clean['property_type'].apply(simplify_property_type)

# The original column is deleted.
df_clean = df_clean.drop(columns='property_type')

# One-Hot Encoding of room_type
col_to_encode = 'room_type'
df_clean = pd.get_dummies(df_clean, columns=[col_to_encode], prefix=col_to_encode, drop_first=False)

# One-Hot Encoding de property_type_simplified
col_to_encode = 'property_type_simplified'
df_clean = pd.get_dummies(df_clean, columns=[col_to_encode], prefix='property_type', drop_first=False)

df_clean.shape

(53086, 39)

## 10. Create neighbourhood prestige groups (`Budget`, `Mid_Range`, `High_End`, `Prime`)

We group boroughs (`neighbourhood_cleansed`) into 4 segments based on median price:
- Budget
- Mid_Range
- High_End
- Prime

Then we one-hot encode this `neigh_group` variable.


In [ ]:
borough_prices = df_clean.groupby('neighbourhood_cleansed')['price'].median().sort_values(ascending=False)

# Median price by neighborhood
neighborhood_stats = df_clean.groupby('neighbourhood_cleansed')['price'].median()

# Division into 4 groups according to quartiles
neighborhood_groups = pd.qcut(neighborhood_stats, q=4, labels=['Budget', 'Mid_Range', 'High_End', 'Prime'])
group_mapping = neighborhood_groups.to_dict()

# Mapping application
df_clean['neigh_group'] = df_clean['neighbourhood_cleansed'].map(group_mapping)

# One-Hot Encoding of neigh_group
col_to_encode = 'neigh_group'
df_clean = pd.get_dummies(df_clean, columns=[col_to_encode], prefix='neigh', drop_first=False)

df_clean.shape


(53086, 43)

## 11. Convert `host_since` to `host_days_active` and remove raw neighbourhood name

We convert the host’s start date into a numeric feature:
- `host_days_active` = number of days the host has been active up to a reference date.

We also drop the original textual columns we no longer need:
- `host_since`
- `neighbourhood_cleansed`


In [ ]:
# Conversion to datetime and creation of host_days_active
df_clean['host_since'] = pd.to_datetime(df_clean['host_since'], errors='coerce')
reference_date = pd.to_datetime('2025-11-20')
df_clean['host_days_active'] = (reference_date - df_clean['host_since']).dt.days
df_clean.drop(columns=['host_since'], inplace=True)

# Removal of detailed neighborhood names, replaced by our groups and dummies
df_clean = df_clean.drop(columns='neighbourhood_cleansed')

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53086 entries, 0 to 96870
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_acceptance_rate            53086 non-null  float64
 1   host_is_superhost               53086 non-null  int64  
 2   host_has_profile_pic            53086 non-null  int64  
 3   host_identity_verified          53086 non-null  int64  
 4   accommodates                    53086 non-null  int64  
 5   bathrooms                       53086 non-null  float64
 6   bedrooms                        53086 non-null  float64
 7   price                           53086 non-null  float64
 8   minimum_nights                  53086 non-null  int64  
 9   maximum_nights                  53086 non-null  int64  
 10  availability_30                 53086 non-null  int64  
 11  availability_365                53086 non-null  int64  
 12  number_of_reviews               53086

## 12. Final sanity checks and filtering unrealistic prices

We re-check the unique values per column and then:

- filter out listings with `price <= 0` or `price > 10000`  
  (considered unrealistic for our business case),
- inspect the new price statistics,
- and finally plot the distribution (zoom on listings under £800).

This produces the final cleaned dataset used in the ML notebook.


In [ ]:
# Quick review of distributions for each column
for column in df_clean.columns:
    unique_values = df_clean[column].unique()

# Filtering out outlier prices
df_clean = df_clean[(df_clean['price'] > 0) & (df_clean['price'] <= 10000)]

df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 53058 entries, 0 to 96870
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_acceptance_rate            53058 non-null  float64
 1   host_is_superhost               53058 non-null  int64  
 2   host_has_profile_pic            53058 non-null  int64  
 3   host_identity_verified          53058 non-null  int64  
 4   accommodates                    53058 non-null  int64  
 5   bathrooms                       53058 non-null  float64
 6   bedrooms                        53058 non-null  float64
 7   price                           53058 non-null  float64
 8   minimum_nights                  53058 non-null  int64  
 9   maximum_nights                  53058 non-null  int64  
 10  availability_30                 53058 non-null  int64  
 11  availability_365                53058 non-null  int64  
 12  number_of_reviews               53058

In [ ]:
chemin_drive = '/content/drive/MyDrive/Projet Machine Learning/Final version/2) Clean Scraped from Airbnb listings.csv'

# Save the DataFrame
df_clean.to_csv(chemin_drive, index=False)